# Dirac Equation on Kerr spacetime + Wald Field

## Basic initialization of the lorentzian manifold

Initialization of the Kerr manifold $\mathcal M_{Kerr}$ atlas with Boyer-Lindquist chart (BL) and of the metric tensor $g$ :

In [1]:
%display latex
from ST_bundle.ST_bundle import ST_bundle

M = Manifold(4, 'M', latex_name=r'\mathcal{M}', structure='Lorentzian')
BL.<t,r,th,ph> = M.chart(r"t r th:(0,pi):\theta ph:(0,2*pi):\phi") #boyer-lindquist chart definition
#SD.<t, r, th, ph> = M.chart(r"t r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi:periodic")
Dc = M.default_chart()
#var('m', domain='real') #definition of Schwarzschild parameters

var('m, a', domain='real') #definition of Kerr metric parameters

g = M.metric() #we put the label g on the metric tensor

rho2 = r^2 + (a*cos(th))^2
Delta = r^2 -2*m*r + a^2

g[0,0] = -(1-2*m*r/rho2)
g[0,3] = -2*a*m*r*sin(th)^2/rho2
g[1,1], g[2,2] = rho2/Delta, rho2
g[3,3] = (r^2+a^2+2*m*r*(a*sin(th))^2/rho2)*sin(th)^2

#g[0, 0] = - (1 - 2*m/r) #Schwarzschild
#g[1, 1] = 1/(1 - 2*m/r)
#g[2, 2] = r^2
#g[3, 3] = r^2*sin(th)^2

g.display()

g = (2*m*r/(a^2*cos(th)^2 + r^2) - 1) dt*dt - 2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) dt*dph + (a^2*cos(th)^2 + r^2)/(a^2 - 2*m*r + r^2) dr*dr + (a^2*cos(th)^2 + r^2) dth*dth - 2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) dph*dt + (2*a^2*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) + a^2 + r^2)*sin(th)^2 dph*dph

## Definition of the orthonormal tetrad and Wald Solution

Definition of the orthonormal tetrad frame $e_{(a)}$ (Chandrasekar notation) and its dual frame $e^{(a)} = \eta^{(a)(b)}g(e_{(b)},-)$ :

In [2]:
#definition of the coframe tetrad fields (as listed in Wald's article)

de0 = M.one_form(sqrt(Delta/rho2), 0, 0, -a*(sin(th)^2)*sqrt(Delta/rho2), frame=Dc.frame(), name='e^{(0)}')
de1 = M.one_form(0, sqrt(rho2/Delta), 0, 0, frame=Dc.frame(), name='e^{(1)}')
de2 = M.one_form(0, 0, sqrt(rho2), 0, frame=Dc.frame(), name='e^{(2)}')
de3 = M.one_form(-(sin(th)/sqrt(rho2))*a, 0, 0, (sin(th)/sqrt(rho2))*(r^2+a^2), frame=Dc.frame(), name='e^{(3)}')

#obtaining the tetrad frame vector fields using musical isomorphism of the metric

e0 = -de0.up(g) #the zero component must be multiplied by a minus because of the lowering of the tetrad index with eta_{(a)(b)}
e1 = de1.up(g)
e2 = de2.up(g)
e3 = de3.up(g)


e = M.vector_frame('e', (e0, e1, e2, e3)) #costruction of the tetrad collection object e_{a}
de = e.coframe() #construction of the tetrad coframe object e^{a}  

Definition of the Wald field solution $A = \frac{B_{0}}{2}(\psi + 2a\eta)$, where $\eta$ and $\psi$ are the 1-forms naturally associated with the musical isomorphism respectively to the two Killing vector fields $\frac{\partial}{\partial t}$ and $\frac{\partial}{\partial \phi}$ :

In [8]:
var('B_0')
var('q')
A_wald = (B_0/2)*(Dc.frame()[3].down(g)+(2*a)*Dc.frame()[0].down(g))
A_wald.set_name("A_{Wald}")
A_wald.display()

A_{Wald} = (B_0*a*m*r - B_0*a*r^2 - (B_0*a^3 - B_0*a*m*r)*cos(th)^2)/(a^2*cos(th)^2 + r^2) dt + 1/2*(2*B_0*a^2*m*r*sin(th)^4 - (4*B_0*a^2*m*r - B_0*a^2*r^2 - B_0*r^4 - (B_0*a^4 + B_0*a^2*r^2)*cos(th)^2)*sin(th)^2)/(a^2*cos(th)^2 + r^2) dph

## Inizialization of the Spinor bundle and of the connections

Initialization of the ST bundle, the Spinor connections $D_{sb}$ and $D_{sb}^{(A_{Wald})}$ and the Dirac matrices $\gamma^{(a)}$:

In [4]:
STb = ST_bundle(M,e)

D_s = STb.sconnection()           #Spinor connection
D_s_A = STb.sconnection(q*A_wald) #Spinor-Wald connection
Gamma = STb.Dirac_matrices("up")      #Dirac Gamma matrices
Gamma.display()

[
[ I  0  0  0]  [ 0  0  0  I]  [ 0  0  0  1]  [ 0  0  I  0]
[ 0  I  0  0]  [ 0  0  I  0]  [ 0  0 -1  0]  [ 0  0  0 -I]
[ 0  0 -I  0]  [ 0 -I  0  0]  [ 0 -1  0  0]  [-I  0  0  0]
[ 0  0  0 -I], [-I  0  0  0], [ 1  0  0  0], [ 0  I  0  0]
]

Definition of a test section of the spinor bundle $\psi = \psi^{i}e_{[s]i}$ :

In [5]:
psi1 = function("psi1", nargs = 4)
psi2 = function("psi2", nargs = 4)
psi3 = function("psi3", nargs = 4)
psi4 = function("psi4", nargs = 4)


psi1_ = M.scalar_field({Dc: psi1(r,t,th,ph)}, name = '\\psi^{(1)}')
psi2_ = M.scalar_field({Dc: psi2(r,t,th,ph)}, name = '\\psi^{(2)}')
psi3_ = M.scalar_field({Dc: psi3(r,t,th,ph)}, name = '\\psi^{(3)}')
psi4_ = M.scalar_field({Dc: psi4(r,t,th,ph)}, name = '\\psi^{(4)}')

psi = STb.sbundle.section({STb.sframe: [psi1_,psi2_,psi3_,psi4_]}, "psi")
psi.display()

psi1(r, t, th, ph) e_0 + psi2(r, t, th, ph) e_1 + psi3(r, t, th, ph) e_2 + psi4(r, t, th, ph) e_3

Calculation of $(\gamma^{(a)})^{i}_{ \ \ j}(D_{sb}(e_{(a)},\psi))^{j}$ (lhs of Dirac's equation on curved spacetime for a massless spinor field) :

In [6]:
Dirac_equation_lhs = Gamma.contract(D_s(e,psi))

Calculation of $(\gamma^{(a)})^{i}_{ \ \ j}(D_{sb}^{(A_{Wald})}(e_{(a)},\psi))^{j}$ (lhs of Dirac's equation on curved spacetime for a massless spinor field coupled to Wald solution $A$) :

In [7]:
Dirac_Wald_equation_lhs = Gamma.contract(D_s_A(e,psi))